In [1]:
from Designer import DesignNotes, SST, Part, show, extract, se_split
import math
sind = lambda deg: math.sin(math.radians(deg))  # return sin of angle expressed in degrees
cosd = lambda deg: math.cos(math.radians(deg))  # return cos of angle expressed in degrees

In [2]:
import pint                  # setup to use the module for computing with units
ureg = pint.UnitRegistry()
mm = ureg['mm']              # define symbols for the units used below
inch = ureg['inch']
kN = ureg['kN']
MPa = ureg['MPa']
ureg.default_format = '~P'

In [3]:
class Bolts(Part):
    'Bolts'   # bolt group is the same on the tongue plate and on the gusset plate.
    grade = 'ASTM A325'
    size = '3/4"'
    Fu = 825*MPa
    d = (3/4*inch).to(mm)
    hole_type = 'punched'
    hd = 22*mm            # hole diameter
    ha = hd + 2*mm        # hole allowance
    threads_intercepted = True
    nlines = 2            # a line is perpendicular to load
    nperline = 3          # number of bolts in each line
    g = 75*mm             # gauge (perpendicular to load)
    s = 75*mm             # spacing (parallel to load)

class Welds(Part):
    'Welds'
    grade = 'E49xx'
    Xu = 490*MPa
    matching = True

class Plates(Part):
    'Plates'
    grade = 'CSA G40.21 350W'
    Fy = 350*MPa
    Fu = 450*MPa

class HSS(Part):
    'HSS Column'
    grade = 'CSA G40.21 350W'
    Fy = 350*MPa
    Fu = 450*MPa
    size = 'HS127x127x13'
    D,T,A = SST.section(size,'D,T,A')
    D = D*mm
    T = T*mm
    A = A*mm*mm    

class CoverPlate(Plates):
    'Cover Plate'
    T = 10*mm
    W = 60*mm
    Lw = 90*mm      # length of weld from net section to end of HSS
    D = 6*mm        # size of weld from on HSS.
   
class TonguePlate(Plates):
    'Tongue Plate'
    T = 20*mm
    W = 280*mm
    L = 260*mm
    e = 40*mm
    D = 8*mm       # weld size
    c = 45*mm      # dist end of weld to 1st bolt line  
    Lw = L - (c + (Bolts.nlines-1)*Bolts.s + e)
    Dh = SST.section(HSS.size,'D')*mm
    ns = 10

class GussetPlate(Plates): 
    'Gusset Plate'
    T,W = TonguePlate.values('T,W')
    W2 = 110*mm
    e = 40*mm    # end distance
    D = 8*mm     # plate to column weld size
    theta = 45. 

In [4]:
c = extract('T,W,L=T*100',TonguePlate,CoverPlate,Plates)
c

Designer.TonguePlate_Extract

In [5]:
c.__ns__()

{T: 20 <Unit('millimeter')>,
 W: 280 <Unit('millimeter')>,
 L: 2000 <Unit('millimeter')>}

In [6]:
TonguePlate.items('T,L,Fy,Dh')

[(T, 20 <Unit('millimeter')>),
 (L, 260 <Unit('millimeter')>),
 (Fy, 350 <Unit('megapascal')>),
 (Dh, 127.0 <Unit('millimeter')>)]

In [7]:
c.show()

L = 2000 mm
T = 20   mm
W = 280  mm


In [8]:
GussetPlate.show()

D     = 8   mm
e     = 40  mm
Fu    = 450 MPa
Fy    = 350 MPa
grade = CSA G40.21 350W 
T     = 20  mm
theta = 45  
W     = 280 mm
W2    = 110 mm


In [9]:
d = GussetPlate.values('e')
d

40 <Unit('millimeter')>

In [10]:
def items(names,*objs,**kwds):
    ns = {}
    for o in reversed(objs):
        if isinstance(o,type):
            ns.update(o.__ns__())
            continue
        if type(o) in (list,tuple):
            if len(o) > 0:
                if type(o[0]) is tuple and len(o[0]) == 2:
                    ns.update(dict(o))
                else:
                    raise ValueError('Invalid object: {}'.format(o))
            continue
        for k,v in vars(o).items():
            if not k.startswith('__'):
                ns[k] = v
    ns.update(kwds)
    
    d = {}
    for expr in se_split(names):
        if '=' in expr:
            target,rhs = [x.strip() for x in expr.split('=',1)]
            if rhs in ns:
                value = ns[rhs]
            else:
                if ns:
                    value = eval(rhs,ns)
                else:
                    raise ValueError("Illegal name: {}={}".format(target,rhs))
        else:
            target = expr
            value = ns[expr] if ns else None
        if target in d:
            raise DesignerError('''Name '{}' has been used more than once.'''.format(target))
        d[target] = value

    return list(d.items())

def values(names,*objs,**kwds):
    return [v for k,v in items(names,*objs,**kwds)]

In [11]:
size = 'HS127x127x13'
s = SST.section(size)
s

In [12]:
items('B=B*mm,T,A=A*mm*mm,s=sin(30)',GussetPlate,s,[('a',1),('b',2)],c=3,a=10,sin=sind,mm=mm)

[(B, 127.0 <Unit('millimeter')>),
 (T, 20 <Unit('millimeter')>),
 (A, 5390.0 <Unit('millimeter ** 2')>),
 (s, 0.49999999999999994)]

In [13]:
items('x,y,z')

[(x, None), (y, None), (z, None)]

In [14]:
class DesignNotes_CM(object):

    """DesignNotes Context Manager."""

    def __init__(self, notes, *objattrs, label=None, result_var=None,
                 local='', locals='', globals='', trace=None, record=True):
        self.notes = notes
        self.objattrs = objattrs
        self.label = label
        if result_var is None:
            result_var = notes.var
        self.result_var = result_var
        self.local = local + locals   # get rid of local some day, locals is preferred because of globals
        self.globals = globals
        if trace is None:
            trace = notes.trace
        self.trace = trace
        self.record = record
        self._setup()

    def _setup(self):
        """Add all attributes/values to the set of global variables.
        Save enough state so that they can be restored when the context
        manager exits."""
        gns = get_ipython().user_ns  # get the ns for the user
        d = {}
        gns = get_ipython().user_ns
        for ob in self.objattrs:
            if ispartial(ob):
                for target,value in ob.ns().items():
                    if target in d:
                        raise DesignerError('''Name '{}' has been used more than once.'''.format(target))
                    d[target] = value
                continue
            else:
                obj,names = ob
            objns = obj.ns()
            for expr in se_split(names):
                if '=' in expr:
                    target,rhs = [x.strip() for x in expr.split('=',1)]
                    if rhs in objns:
                        value = objns[rhs]
                    else:
                        value = eval(rhs,gns,objns)
                else:
                    target = expr
                    value = objns[expr]
                if target in d:
                    raise DesignerError('''Name '{}' has been used more than once.'''.format(target))
                d[target] = value
        self.new_values = d
        self.local_vars = [y for y in [x.strip() for x in self.local.split(',')] if y] + [self.result_var]
        self.global_vars = [y for y in [x.strip() for x in self.globals.split(',')] if y]
        
        self.changed_values = {}
        self.added_vars = []
        
    def __enter__(self):
        gns = get_ipython().user_ns
        for k,v in self.new_values.items():
            if k in gns:
                self.changed_values[k] = gns[k]
            else:
                self.added_vars.append(k)
            gns[k] = v
            
        for k in self.local_vars:
            if k in self.changed_values or k in self.added_vars:
                raise DesignerError('''Name '{}' has been used more than once.'''.format(k))
            if k in gns:
                self.changed_values[k] = gns[k]
                del gns[k]
            else:
                self.added_vars.append(k)
        return self
    
    enter = __enter__
    
    def __exit__(self,exc_type,exc_value,exc_tb):
        """When the context exits, restore the global values to what they
        were before entering."""
        gns = get_ipython().user_ns  # get the ns for the user
        
        # capture current values of all local variables
        dct = {}
        for k,v in self.changed_values.items():
            if k in gns:
                dct[k] = gns[k]
        for k in self.added_vars + self.global_vars:
            if k in gns:
                dct[k] = gns[k]
        self.ending_values = dct
        
        # record the result produced
        if self.record and self.label and exc_type is None:
            rkey = self.result_var
            if rkey in gns:
                rval = gns[rkey]
            else:
                raise DesignerError('''Result variable '{}' is not defined.'''.format(rkey))
            self.notes.record(rval,self.label,values=self.ending_values)
        
        # now change the user ns back to what it was before __enter__
        for k,v in self.changed_values.items():
            gns[k] = v               # restore old values
        for k in self.added_vars:
            if k in gns:
                del gns[k]           # or delete them if they were newly created
        self.changed_values = {}
        self.added_vars = []
        return False              # to re-raise exceptions
    
    exit = __exit__


In [57]:
class DesignNotes(object):
    
    def __init__(self,var,trace=False,units=None,selector=min,title='',nsigfigs=3):
        self.trace = trace
        self.var = var
        self.units = units
        self.selector = selector
        self.title = title
        self.nsigfigs = nsigfigs
        self._notes = []
        self._checks = []
        self._record = []
        self._execution_count = None
        
    def makeVARS(self,trace=None):
        if trace is None:
            trace = self.trace
        def _makeVARS(varslist,*objlist,**kwds):
            nonlocal trace
            nonlocal self
            return CM1(self,items(varslist,*objlist,**kwds),trace=trace)
        return _makeVARS
            
    def makeREC(self,trace=None):
        if trace is None:
            trace = self.trace
        def _makeVARS(var,title,**kwds):
            nonlocal trace
            nonlocal self
            return CM1(self,[(var,None)],trace=trace,title=title)
        return _makeVARS
 

In [63]:
class CM1(object):
    
    def __init__(self,notes,itemlist,trace=False,title=''):
        self.notes = notes
        self.itemlist = itemlist
        self.trace = trace
        self.title = title
        self.changed_vars = {}
        self.added_vars = []
        self.gns = get_ipython().user_ns
    
    def __enter__(self):
        for k,v in self.itemlist:
            if k in self.gns:
                self.changed_vars[k] = self.gns[k]
            else:
                self.added_vars.append(k)
            self.gns[k] = v
        
    def __exit__(self,*args):
        if self.trace:
            values = {}
            for k,v in self.itemlist:
                values[k] = self.gns[k]
            if values:
                show(','.join([k for k,v in values.items()]),data=values,minwidth=5)
        for k in self.added_vars:
            if k in self.gns:
                del self.gns[k]
        self.added_vars = []
        for k,v in self.changed_vars.items():
            self.gns[k] = v
        self.changed_vars = {}
        

In [64]:
dn = DesignNotes('xx',trace=True)
REC = dn.makeREC()
VARS = dn.makeVARS()

In [66]:
phiw = 0.67
with REC('Vr','Gusset to HSS Weld'), \
     VARS('W,W2,theta,D',GussetPlate), \
     VARS('Xu',Welds), \
     VARS('L,L1,Mw,Aw'):
    L1 = W2+W*cosd(theta)
    L = (L1/sind(theta))*cosd(theta) + W*sind(theta)

    Mw = 1.0                  
    Aw = 2*L*.707*D
    Vr = 0.67*phiw*Aw*Xu*(1+sind(theta)**1.5)*Mw     # S16-14: 13.13.2.2
    Vr = Vr.to(kN)

L     = 506  mm
L1    = 308  mm
Mw    = 1    
Aw    = 5724 mm²
Xu    = 490 MPa
W     = 280 mm
W2    = 110 mm
theta = 45  
D     = 8   mm
Vr    = 2008 kN


In [ ]:
L